
# Prepare glacier outlines + center-lines for submission to GLIMS
#### Author: Wilson Cheung, May 2025
 

In [ ]:
import geopandas as gpd
from pathlib import Path
from shapely.geometry import MultiPolygon
from shapely.ops import unary_union
from shapely.geometry.polygon import orient
from shapely.validation import make_valid

In [ ]:
# USER INPUT BLOCK  ✏️
glacier_name = "Turner"  # name of the glacier

# input folders
base_dir   = Path("dummy_data")   # outlines
center_dir = Path("dummy_data")   # centre-lines

# output folder (will be created if absent)
out_dir    = Path("dummy_output") / glacier_name
out_dir.mkdir(parents=True, exist_ok=True)

# file paths
outline60_fp = base_dir   / f"{glacier_name}1960.shp"
outline23_fp = base_dir   / f"{glacier_name}2023.shp"
cl60_fp      = center_dir / f"{glacier_name}1960_Centerlines_smooth.shp"
cl23_fp      = center_dir / f"{glacier_name}2023_Centerlines_smooth.shp"

meta1960 = dict(orig_id="A16817", acq_time="1960-09-06", inst_name="Historical Aerial Photo from  National Air Photo Library (NAPL)")
meta2023 = dict(orig_id="20230817_154317_42_2424", acq_time="2023-08-17", inst_name="PlanetScope Satellite Imagery")

analysis_info = dict(
    analyst_name="Wilson (Wai Yin) Cheung",
    digitize_date="2023-09-20",
    method_desc="Manual digitization of glacier outlines using ArcGIS Pro, followed the guidelines established by Paul et al. (2017)"
)

# 1960 uncertainty 
UNC60_LOC  = 2    # metres   
UNC60_GLOB = 0.05

# 2023 uncertainty (PlanetScope scene)
UNC23_LOC  = 1.5     # metres
UNC23_GLOB = 7

In [ ]:
def read_vector(path):                            # read + sanity CRS
    gdf = gpd.read_file(path)
    if gdf.crs is None:
        raise ValueError(f"{path} has no CRS defined.")
    return gdf

def fix_polygons(gdf):                            # validity + CCW
    repaired = []
    for geom in gdf.geometry:
        if not geom.is_valid:
            geom = make_valid(geom)
        parts = geom.geoms if isinstance(geom, MultiPolygon) else [geom]
        repaired.append(unary_union([orient(p, sign=1.) for p in parts]))
    gdf = gdf.copy(); gdf.geometry = repaired
    return gdf

def to_wgs84(gdf):
    return gdf.to_crs(4326) if gdf.crs.to_epsg() != 4326 else gdf

def _fill(gdf, col, value):
    """Create col or replace NaN; keep existing non-NaN."""
    if col not in gdf.columns:
        gdf[col] = value
    else:
        gdf[col] = gdf[col].fillna(value)

def add_attrs(gdf, meta, loc_unc_val, glob_unc_val):
    gdf = gdf[['geometry']].copy()  # Keep only the geometry column
    for c in ("loc_unc_x", "loc_unc_y"):   _fill(gdf, c, loc_unc_val)
    for c in ("glob_unc_x","glob_unc_y"):  _fill(gdf, c, glob_unc_val)
    gdf["glacier_name"]       = glacier_name
    gdf["orig_id"]    = meta["orig_id"]
    gdf["acq_time"]   = meta["acq_time"]
    gdf["inst_name"]  = meta["inst_name"]
    gdf["analyst"]       = analysis_info["analyst_name"]
    gdf["digitize_date"] = analysis_info["digitize_date"]
    gdf["method"]        = analysis_info["method_desc"]
    return gdf

def prep_centerline(path, meta):
    cl = to_wgs84(read_vector(path))
    cl = cl[['geometry']].copy()  # keep only the geometry column
    cl["glacier_name"] = glacier_name
    cl["line_type"]    = "centerline"
    cl["inst_name"]    = meta["inst_name"]
    cl["author"]       = "Wilson (Wai Yin) Cheung"
    cl["method"]       = "The Open Global Glacier Model (OGGM) v1.6"
    return cl

In [ ]:
# Read and print attribute columns for each shapefile
gdf_outline60 = read_vector(outline60_fp)
print("Attributes in outline60_fp:")
print(gdf_outline60.columns.tolist())

gdf_outline23 = read_vector(outline23_fp)
print("\nAttributes in outline23_fp:")
print(gdf_outline23.columns.tolist())

gdf_cl60 = read_vector(cl60_fp)
print("\nAttributes in cl60_fp:")
print(gdf_cl60.columns.tolist())

gdf_cl23 = read_vector(cl23_fp)
print("\nAttributes in cl23_fp:")
print(gdf_cl23.columns.tolist())

In [ ]:
# Visualize outlines and centerlines
ax = gdf_outline60.plot(edgecolor='blue', facecolor='none', figsize=(8,8), label='1960 outline')
gdf_outline23.plot(ax=ax, edgecolor='red', facecolor='none', label='2023 outline')
gdf_cl60.plot(ax=ax, color='cyan', label='1960 centerline')
gdf_cl23.plot(ax=ax, color='magenta', label='2023 centerline')
ax.legend()
ax.set_title('Dummy glacier data')

In [ ]:
# PROCESS
print("Loading input layers …")
gdf60_raw = read_vector(outline60_fp)
gdf23_raw = read_vector(outline23_fp)

print("Repairing geometry & reprojecting …")
gdf60 = add_attrs(to_wgs84(fix_polygons(gdf60_raw)), meta1960,
                  loc_unc_val=UNC60_LOC,  glob_unc_val=UNC60_GLOB)
gdf23 = add_attrs(to_wgs84(fix_polygons(gdf23_raw)), meta2023,
                  loc_unc_val=UNC23_LOC,  glob_unc_val=UNC23_GLOB)

cl60  = prep_centerline(cl60_fp, meta1960)
cl23  = prep_centerline(cl23_fp, meta2023)

In [ ]:
# EXPORT
out60_fp = out_dir / f"{glacier_name}1960.shp"
out23_fp = out_dir / f"{glacier_name}2023.shp"
cl60_out = out_dir / f"{glacier_name}1960_centerline.shp"
cl23_out = out_dir / f"{glacier_name}2023_centerline.shp"

gdf60.to_file(out60_fp); print("✔ wrote", out60_fp)
gdf23.to_file(out23_fp); print("✔ wrote", out23_fp)
cl60 .to_file(cl60_out); print("✔ wrote", cl60_out)
cl23 .to_file(cl23_out); print("✔ wrote", cl23_out)

print("\nAll four layers are GLIMS-ready and stored in", out_dir)

In [ ]:
# Read output shapefiles
gdf_out_1960 = gpd.read_file(out60_fp)
gdf_out_2023 = gpd.read_file(out23_fp)
gdf_cl_out_1960 = gpd.read_file(cl60_out)
gdf_cl_out_2023 = gpd.read_file(cl23_out)

print("Columns for the 1960 outline output:")
print(gdf_out_1960.columns.tolist())

print("\nColumns for the 2023 outline output:")
print(gdf_out_2023.columns.tolist())

print("\nColumns for the 1960 centerline output:")
print(gdf_cl_out_1960.columns.tolist())

print("\nColumns for the 2023 centerline output:")
print(gdf_cl_out_2023.columns.tolist())